In [1]:
import copy
import itertools as it
import numpy as np
import yaml
import os

def expand_config(dict_config):
    keys, values = zip(*dict_config.items())
    permutations_dicts = [dict(zip(keys, v)) for v in it.product(*values)]
    return permutations_dicts

In [5]:
def generate_train_str_config(config):
    config_list = []
    config_list.append('ignore_exceptions=False use_density_based_ue=True')
    
    config_list.append('batch_size={}'.format(config['batch_size']))

    if "gsm8k" in config["task"]:
        config_list.append('subsample_train_dataset={}'.format(config['subsample_gsm8k_train_dataset']))
    else:
        config_list.append('subsample_train_dataset={}'.format(config['subsample_train_dataset']))
    config_list.append('subsample_background_train_dataset={}'.format(config['subsample_background_train_dataset']))
    config_list.append('subsample_eval_dataset={}'.format(config['subsample_eval_dataset']))
    config_list.append('model.path={}'.format(config['model']))
    
    if ("gemma" in config['model']) or ("mistral" in config['model'].lower()) or ("llama-3" in config['model'].lower()) or ("stablelm-2" in config['model'].lower()):
        config_list.append('+model.attn_implementation=eager')
    if ("cache_path" in config.keys()):
        config_list.append('cache_path={}'.format(config['cache_path']))
    if ("metric_thrs" in config.keys()):
        config_list.append('+metric_thrs={}'.format(config['metric_thrs']))
    if ("layers" in config.keys()):
        config_list.append('+layers={}'.format(config['layers'][config['model']]))
    if ("samples_n" in config.keys()):
        config_list.append('+generation_params.samples_n={}'.format(config['samples_n']))
    return config_list

In [6]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../'
    j = 0
    n_devices = len(cuda_devices)
    for i, mc_configs in enumerate(configs):
        for conf in expand_config(mc_configs):
            for task_name in tasks:
                conf["task"] = task_name
                if not ("12b" in conf["model"]):
                    base_arg = f'CUDA_VISIBLE_DEVICES={cuda_devices[j%n_devices]} HYDRA_CONFIG=./configs/polygraph_eval_{task_name}.yaml python run_polygraph_2.py'
                else:
                    base_arg = f'CUDA_VISIBLE_DEVICES={cuda_devices[j%n_devices]},{cuda_devices[(j+1)%n_devices]} HYDRA_CONFIG=./configs/polygraph_eval_{task_name}.yaml python run_polygraph_2.py'
                    j+=1
                    
                new_task = copy.deepcopy(base_arg)
                args = ' '.join(generate_func[i](conf))
                new_task += f' {args}'
                if (j+1)%n_devices!=0: 
                    new_task += ' &'
                else:
                    new_task += '\nwait'
                full_config += '\n' + new_task if len(full_config) else new_task
                j+=1
                
    with open (f'../scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [7]:
tasks = ['coqa', 'sciq', 'triviaqa', 'truthfullqa', 'mmlu']

layers = {"google/gemma-7b": "\"[{},-1]\"".format(",".join([str(x) for x in range(27)])),
          "meta-llama/Meta-Llama-3-8B": "\"[{},-1]\"".format(",".join([str(x) for x in range(31)])),
          "stabilityai/stablelm-2-12b": "\"[{},-1]\"".format(",".join([str(x) for x in range(39)]))}

train_configs = {
    'batch_size': [1],
    'subsample_train_dataset': [1000],
    'subsample_background_train_dataset': [1000],
    'subsample_eval_dataset': [2000],
    'model': ['google/gemma-7b', 'meta-llama/Meta-Llama-3-8B'],# 'stabilityai/stablelm-2-12b'],
    'metric_thrs': ["\"[0.3]\""],
    'layers': [layers],
    'cache_path': ['./workdir/output_layers_9'],
    'samples_n': [5],
}
    
cuda_devices = [0]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_tmd_exps_p1.sh')

In [8]:
tasks = ['gsm8k', 'xsum', 'wmt14_deen', 'medquad', 'cnn', 'samsum', 'pubmedqa']

train_configs = {
    'batch_size': [1],
    'subsample_train_dataset': [1000],
    'subsample_background_train_dataset': [500],
    'subsample_eval_dataset': [2000],
    'model': ['google/gemma-7b', 'meta-llama/Meta-Llama-3-8B'],# 'stabilityai/stablelm-2-12b'],
    'metric_thrs': ["\"[0.3]\""],
    'layers': [layers],
    'cache_path': ['./workdir/output_layers_9'],
    'samples_n': [5],
}
    
cuda_devices = [0,1]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_tmd_exps_p2.sh')

In [4]:
tasks = ['sciq', 'coqa', 'triviaqa', 'truthfullqa', 'mmlu', 'samsum', 'pubmedqa', 'wmt14_deen', 'cnn']

layers = {"google/gemma-7b": "\"[{},-1]\"".format(",".join([str(x) for x in range(27)])),
          "meta-llama/Meta-Llama-3-8B": "\"[{},-1]\"".format(",".join([str(x) for x in range(31)])),
          "stabilityai/stablelm-2-12b": "\"[{},-1]\"".format(",".join([str(x) for x in range(39)]))}

train_configs = {
    'batch_size': [1],
    'subsample_train_dataset': [5000],
    'subsample_background_train_dataset': [2000],
    'subsample_eval_dataset': [2000],
    'model': ['google/gemma-7b', 'meta-llama/Meta-Llama-3-8B'],
    'metric_thrs': ["\"[0.3]\""],
    'layers': [layers],
    'cache_path': ['./workdir/output_layers_3'],
    'samples_n': [5],
}
    
cuda_devices = [0]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_tmd_exps_p1_1.sh')

In [5]:
tasks = ['gsm8k', 'xsum', 'medquad']

train_configs = {
    'batch_size': [1],
    'subsample_train_dataset': [2000],
    'subsample_background_train_dataset': [2000],
    'subsample_eval_dataset': [2000],
    'model': ['google/gemma-7b', 'meta-llama/Meta-Llama-3-8B'],
    'metric_thrs': ["\"[0.3]\""],
    'layers': [layers],
    'cache_path': ['./workdir/output_layers_3'],
    'samples_n': [5],
}
    
cuda_devices = [1,2]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_tmd_exps_p2_1.sh')

In [6]:
tasks = ['sciq', 'coqa', 'triviaqa', 'truthfullqa', 'mmlu', 'samsum', 'pubmedqa', 'wmt14_deen', 'gsm8k', 'xsum', 'medquad', 'cnn']

train_configs = {
    'batch_size': [1],
    'subsample_train_dataset': [5000],
    'subsample_background_train_dataset': [2000],
    'subsample_eval_dataset': [2000],
    'model': ['stabilityai/stablelm-2-12b'],
    'metric_thrs': ["\"[0.3]\""],
    'layers': [layers],
    'cache_path': ['./workdir/output_layers_3'],
    'samples_n': [5],
}
    
cuda_devices = [1,2]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_tmd_exps_p3_1.sh')

In [7]:
tasks = ['pubmedqa', 'wmt14_deen', 'cnn']

layers = {"google/gemma-7b": "\"[{},-1]\"".format(",".join([str(x) for x in range(27)])),
          "meta-llama/Meta-Llama-3-8B": "\"[{},-1]\"".format(",".join([str(x) for x in range(31)])),
          "stabilityai/stablelm-2-12b": "\"[{},-1]\"".format(",".join([str(x) for x in range(39)]))}

train_configs = {
    'batch_size': [1],
    'subsample_train_dataset': [5000],
    'subsample_background_train_dataset': [2000],
    'subsample_eval_dataset': [2000],
    'model': ['meta-llama/Meta-Llama-3-8B'],
    'metric_thrs': ["\"[0.3]\""],
    'layers': [layers],
    'cache_path': ['./workdir/output_layers_3'],
    'samples_n': [5],
}
    
cuda_devices = [0]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_tmd_exps_p1_2.sh')

In [8]:
tasks = ['gsm8k', 'xsum', 'medquad', 'cnn']

train_configs = {
    'batch_size': [1],
    'subsample_train_dataset': [5000],
    "subsample_gsm8k_train_dataset": [2000],
    'subsample_background_train_dataset': [2000],
    'subsample_eval_dataset': [2000],
    'model': ['stabilityai/stablelm-2-12b'],
    'metric_thrs': ["\"[0.3]\""],
    'layers': [layers],
    'cache_path': ['./workdir/output_layers_3'],
    'samples_n': [5],
}
    
cuda_devices = [1,2]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_tmd_exps_p3_2.sh')